In [1]:
#import
import pandas as pd
import numpy as np
import requests

In [2]:
forex_data = pd.read_feather('/kaggle/input/economic-calendar-data/economic_data.feather')

In [3]:
forex_data

timestamp                          title country  \
0       1358933400             Inflation Rate YoY      CO   
1       1358933400             Inflation Rate MoM      CO   
2       1358933400            Total Vehicle Sales      US   
3       1358933400             Inflation Rate YoY      EG   
4       1358933400        Core Inflation Rate YoY      EG   
...            ...                            ...     ...   
189830  1706281200         Pending Home Sales YoY      US   
189831  1706284800                 Budget Balance      CA   
189832  1706292000     Baker Hughes Oil Rig Count      US   
189833  1706292000  Baker Hughes Total Rigs Count      US   
189834  1706319000   Industrial Profits (YTD) YoY      CN   

                      indicator  \
0                Inflation Rate   
1            Inflation Rate Mom   
2           Total Vehicle Sales   
3                Inflation Rate   
4           Core Inflation Rate   
...                         ...   
189830       Pending Home Sales   
189831  Government Budget Value   
189832           Crude Oil Rigs   
189833               Total Rigs   
189834        Corporate Profits   

                                                  comment  actual  previous  \
0       In Colombia, the three main components of the ...    3.18      3.27   
1       Inflation Rate MoM measures month over month c...    0.30      0.12   
2       In the United States, Total Vehicle Sales meas...   16.70     17.55   
3       In Egypt, the headline Consumer Price Index (C...   12.70     12.00   
4       In Egypt, the core inflation rate tracks chang...    8.60      8.30   
...                                                   ...     ...       ...   
189830  The Pending Home Sales Index, a leading indica...    1.30     -5.40   
189831  The government budget balance is the differenc...   -4.01     -6.33   
189832  US Crude Oil Rigs refer to the number of activ...  499.00    497.00   
189833  US Total Rigs refer to the number of active US...  621.00    620.00   
189834  In China, corporate profits refer to total pro...   -2.30     -4.40   

        forecast  importance                      date  
0           3.20          -1 2013-01-23 09:30:00+00:00  
1           0.32          -1 2013-01-23 09:30:00+00:00  
2          17.20           0 2013-01-23 09:30:00+00:00  
3            NaN          -1 2013-01-23 09:30:00+00:00  
4            NaN          -1 2013-01-23 09:30:00+00:00  
...          ...         ...                       ...  
189830       NaN           0 2024-01-26 15:00:00+00:00  
189831       NaN          -1 2024-01-26 16:00:00+00:00  
189832       NaN          -1 2024-01-26 18:00:00+00:00  
189833       NaN          -1 2024-01-26 18:00:00+00:00  
189834       NaN          -1 2024-01-27 01:30:00+00:00  

[189835 rows x 10 columns]

In [ ]:
economic_poland = economic_df[economic_df['country']=='PL'].copy()
economic_poland.reset_index(inplace=True)
economic_poland.drop('index', inplace=True, axis=1)

In [ ]:
# fix some data(sometimes records from previous months are saved next month as duplicates)
economic_poland = economic_df[economic_df['country']=='PL'].copy()

mask = economic_poland.duplicated(subset=['date', 'title'], keep=False)
for idx, [index, row] in enumerate(economic_poland.loc[mask].iterrows()):
    try:
        pair = economic_poland.loc[mask].iloc[idx+1]
        if all(row[['date', 'title']] == pair[['date', 'title']]):
            if row['actual'] == pair['previous']:
                new_date = pd.to_datetime(row['date'])
                time_diff = pd.to_datetime(economic_poland[economic_poland['title'] == row['title']]['date']).diff().dt.days.median()
                new_date = new_date - pd.Timedelta(days=time_diff)
                economic_poland.loc[[index], ['date']] = new_date.strftime('%Y-%m-%d')
            else:
                new_date = pd.to_datetime(pair['date'])
                time_diff = pd.to_datetime(economic_poland[economic_poland['title'] == pair['title']]['date']).diff().dt.days.median()
                new_date = new_date - pd.Timedelta(days=time_diff)
                economic_poland.loc[[pair.name], ['date']] = new_date.strftime('%Y-%m-%d')
    except Exception as e:
        continue
        #print(e)

In [ ]:
#Create new df with continues range of dates and all indicator values in any day

df_pivot = economic_poland.pivot(columns='title', values='actual')

#merge tables to update dates
merged = df_pivot.merge(economic_poland, left_index=True, right_index=True)
merged.drop(['title', 'country', 'indicator', 'comment','actual', 'previous', 'forecast', 'importance'], axis=1, inplace=True)
merged.reset_index(inplace=True)

#create continues dates from oldest to newest 
idx = pd.date_range(merged.date.min(), merged.date.max())
idx = idx.strftime('%Y-%m-%d')

#create new dataframe with full set of date range
new_df = pd.DataFrame(index=idx, columns=merged.columns)
new_df.drop('date', axis=1, inplace=True)

# change date column to str to be merge
merged['date'] = merged['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

#merge both dataframes table with values in full set of ranges
fullset = pd.merge(new_df, merged, how='left', right_on='date', left_index=True, suffixes=("_x", None))
fullset.dropna(axis=1, how='all', inplace=True)

#fill all nans with values from previous rows(newest)
fullset = fullset.ffill()

#drop duplicates
fullset.drop_duplicates(subset=['date'], inplace=True, keep='last')
fullset.set_index('date', inplace=True)
fullset.drop('index', inplace=True, axis=1)

#fill rest of nan values(oldest data) with oldest 'previous' value from main df - check if it is not bettter to leave nan
for col in fullset.columns:
    if pd.isna(fullset[col].iloc[0]):
        value = economic_poland.loc[economic_poland['title'] == col].iloc[0]['previous']
        fullset[col] = fullset[col].fillna(value)
        
#check if last row in our dataframe is correct
test_df = pd.DataFrame()
for title in economic_poland['title'].unique():
    test_df[title] = [economic_poland[economic_poland["title"]==title].iloc[-1]["actual"]]
    
test_true = fullset.drop('timestamp', axis=1).iloc[-1] == test_df
print(test_true.iloc[0].unique()) #it should only contain "True" values

In [ ]:
error = []
for title in economic_poland.title.unique():
    all_rows = economic_poland[economic_poland.title == title]
    for index, [idx, row] in enumerate(all_rows.iterrows()):
        try:
            now = row['date']
            until = pd.to_datetime(all_rows.iloc[index+1]['date'])
            until = until - pd.Timedelta(days=1)
            until = until.strftime('%Y-%m-%d')
        except:
            pass
        if now > until:
            temp1 = now
            now = until
            until = temp1
        actual = row['actual']
        if all(actual != fullset.loc[now:until][indicator]):
            error.append([now, until, actual, fullset.loc[now][indicator], indicator])

In [ ]:
#merge both dataframes table with values in full set of ranges
fullset2 = pd.merge(new_df, merged, how='left', right_on='date', left_index=True, suffixes=("_x", None))
fullset2.dropna(axis=1, how='all', inplace=True)
fullset2.set_index('date', inplace=True)

In [ ]:
fullset['Inflation Rate YoY Final']['2016-03-01':'2016-03-30']

In [ ]:
mask = economic_poland.duplicated(subset=['date', 'title'], keep=False)
economic_poland.loc[mask]

In [ ]:
economic_poland[economic_poland['date'] == '2016-03-15']

In [ ]:
fullset2['Inflation Rate YoY Final'].dropna()[0:20]

In [ ]:
fullset2['Inflation Rate YoY Final']['2016-03-01':'2016-03-30']

In [ ]:
#Forex data
forex_df = pd.read_excel('/kaggle/input/forex-data-gatherer/FOREX_DATA.xlsx')

forex_df['Datetime'] = pd.to_datetime(forex_df['Datetime'], dayfirst=True)
forex_df = dataframe.sort_values(by='Datetime', ascending=False)
forex_df['Datetime'] = forex_df["Datetime"].dt.strftime('%d-%m-%Y %H:%M:%S %z')
forex_df.set_index('Datetime', inplace=True)

In [ ]:
join_df = dataframe.join(calendar_df).drop_duplicates()
join_df = join_df[~join_df.index.duplicated(keep='first')]

In [ ]:
join_df.loc['28-03-2023 07:00:00 +0100']